In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torchvision import datasets, transforms, models
from tqdm import tqdm


In [3]:
trainSet = pd.read_csv('Dataset/train.csv')
classes = trainSet[['id','species']].copy()
classes['id'] = classes['id'].astype(str)
classes['label'] = LabelEncoder().fit_transform(classes['species'])

image_folder = 'Dataset/images/'
imgs = []
labels = []
for i in sorted(os.listdir(image_folder)):
    id = i.split('.')[0]
    if id in classes['id'].values:
        labels.append(classes[classes['id'] == id]['label'].values[0])
        image = Image.open(os.path.join(image_folder, i)).convert('1')
        imgs.append(image)

print(f"There are {len(imgs)} images in the dataset")
print(f"There are {len(np.unique(labels))} labels in the dataset")


There are 990 images in the dataset
There are 99 labels in the dataset


In [4]:

X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.2, random_state=42, stratify=labels)
print(f"There are {len(X_train)} images in the training set")
print(f"There are {len(X_test)} images in the test set")
print(f"There are {len(np.unique(y_train))} classes in the training set")
print(f"There are {len(np.unique(y_test))} classes in the test set")


There are 792 images in the training set
There are 198 images in the test set
There are 99 classes in the training set
There are 99 classes in the test set


In [5]:
class CustomDataSet(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.transform(image=np.array(self.images[index],dtype=np.float32))['image']
        label = torch.tensor(self.labels[index], dtype=torch.long)
        return image, label


    
transform = A.Compose([
    A.Resize(224, 224),
    ToTensorV2()
])

trainDataSet = CustomDataSet(images=X_train, labels=y_train, transform=transform)
testDataSet = CustomDataSet(images=X_test, labels=y_test, transform=transform)




In [7]:
batch_size = 64
trainDataLoader = torch.utils.data.DataLoader(trainDataSet, batch_size=batch_size)
testDataLoader = torch.utils.data.DataLoader(testDataSet, batch_size=batch_size)

min_loss_epoch = 0
min_loss_value = -1
best_model_weights_paths = {}

best_val_loss = float('inf')  # Initialize with a large value
best_epoch = -1
best_model_weights = None
train_losses = []
val_losses = []

criterion = nn.CrossEntropyLoss()
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 99)

optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 1000

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(trainDataLoader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch"):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(trainDataLoader) / batch_size
    train_losses.append(train_loss)
    
    # Validation loop
    model.eval()  
    correct_predictions = 0
    total_samples = 0
    with torch.no_grad():
        for images, labels in testDataLoader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
    accuracy = correct_predictions / total_samples
    val_loss = 1-accuracy
    val_losses.append(val_loss)
    print(f"Epoch: {epoch+1}/{num_epochs}  Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")


best_epoch = val_losses.index(min(val_losses))
min_loss_epoch = best_epoch
min_loss_value = f'{min(val_losses):.4f}'
print(f"Min Train Loss: {min(train_losses)} at Epoch {train_losses.index(min(train_losses))}  Min Val Loss: {min_loss_value[criterion]} at Epoch {best_epoch}")


# Save the best model's weights
torch.save(model, f"best_model.pth")


/Users/youssefgeorge/.pyenv/versions/3.9.18/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/youssefgeorge/.pyenv/versions/3.9.18/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/1000: 100%|██████████| 13/13 [00:48<00:00,  3.77s/batch]


Epoch: 1/1000  Loss: 0.0567, Val Loss: 0.9596


Epoch 2/1000: 100%|██████████| 13/13 [00:50<00:00,  3.85s/batch]


Epoch: 2/1000  Loss: 0.0236, Val Loss: 0.8990


Epoch 3/1000: 100%|██████████| 13/13 [00:50<00:00,  3.85s/batch]


...

Epoch: 41/1000  Loss: 0.0000, Val Loss: 0.1010


Epoch 42/1000: 100%|██████████| 13/13 [00:49<00:00,  3.80s/batch]


Epoch: 42/1000  Loss: 0.0000, Val Loss: 0.1010


Epoch 43/1000:   8%|▊         | 1/13 [00:09<01:48,  9.01s/batch]


KeyboardInterrupt: 